In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout],[pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10]
pin = 0.5
pout = 0.01
seed = 0
alpha = 1e-3
lamda = 1e-3
eta = 1e-3
no_users = sum(cluster_sizes)
batch_size = 20
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.         0.         0.         0.125      0.
  0.         0.16666667 0.2        0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.16666667]
 [0.         0.         0.         0.14285714 0.125      0.14285714
  0.         0.14285714 0.14285714 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.14285714]
 [0.         0.         0.         0.14285714 0.         0.14285714
  0.         0.16666667 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.14285714 0.14285714 0.         0.125      0.14285714
  0.14285714 0.14285714 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.125      0.125      0.         0.125      0.         0.125
  0.125      0.125      0.         0.         0.         0.
  0.         0.         0.       

In [4]:
def load_dataset():
    transforms_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])
    mnist_data_train = datasets.MNIST('./data/mnist', train=True, download=True, transform=transforms_mnist)
    mnist_data_test = datasets.MNIST('./data/mnist', train=False, download=True, transform=transforms_mnist)

    return mnist_data_train, mnist_data_test

In [5]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [6]:
datapoints = {}
count = 0
W1 = np.array([2.0, 2.0])
W2 = np.array([-2.0, 2.0])
W = [W1, W2]
m = 200
n = 2
noise_sd = 0.001
for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
        label = np.dot(features, W[i]) + np.random.normal(0,noise_sd)
        datapoints[count] = {
                'features': features,
                'degree': node_degree(count, G),
                'label': label,
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i])
            }
        count += 1

In [7]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [8]:
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(2, 1, bias=False)
        #self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        #x = F.relu(self.fc1(x))
        output = self.fc1(x)
        #output = self.fc3(x)
        return output

In [9]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [10]:
model = MLP_Net(user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=50, shuffle=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for (x, y) in dataloader:
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)
        print(y.size())
        print(yhat.size())
        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss, grads_to_vector(model.parameters()), parameters_to_vector(model.parameters()))
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(5.4179, grad_fn=<MseLossBackward0>) tensor([ 4.2566, -1.9168]) tensor([-0.0802,  0.6100], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(4.3026, grad_fn=<MseLossBackward0>) tensor([ 3.0161, -2.1468]) tensor([-0.1227,  0.6292], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(3.7889, grad_fn=<MseLossBackward0>) tensor([ 2.9180, -1.6215]) tensor([-0.1529,  0.6507], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(5.0133, grad_fn=<MseLossBackward0>) tensor([ 3.2266, -3.1217]) tensor([-0.1821,  0.6669], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(4.7033, grad_fn=<MseLossBackward0>) tensor([ 3.9569, -1.7984]) tensor([-0.2143,  0.6981], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(3.7402, grad_fn=<MseLossBackward0>) tensor([ 2.8045, -2.0120]) tensor([-0.2539,  0.7161], grad_fn=<CatBackward0>)
torch.Size([50, 1])
to

19 tensor(0.2634, grad_fn=<MseLossBackward0>) tensor([ 0.7238, -0.4863]) tensor([-1.5312,  1.6145], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
19 tensor(0.3583, grad_fn=<MseLossBackward0>) tensor([ 0.8256, -0.8822]) tensor([-1.5385,  1.6193], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
20 tensor(0.3233, grad_fn=<MseLossBackward0>) tensor([ 0.9943, -0.5273]) tensor([-1.5467,  1.6282], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
20 tensor(0.2633, grad_fn=<MseLossBackward0>) tensor([ 0.7081, -0.5800]) tensor([-1.5567,  1.6334], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
20 tensor(0.2290, grad_fn=<MseLossBackward0>) tensor([ 0.6728, -0.4560]) tensor([-1.5637,  1.6392], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
20 tensor(0.3120, grad_fn=<MseLossBackward0>) tensor([ 0.7687, -0.8251]) tensor([-1.5705,  1.6438], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
21 tensor(0.2809

39 tensor(0.0188, grad_fn=<MseLossBackward0>) tensor([ 0.1807, -0.1645]) tensor([-1.8863,  1.8968], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
39 tensor(0.0162, grad_fn=<MseLossBackward0>) tensor([ 0.1688, -0.1329]) tensor([-1.8881,  1.8984], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
39 tensor(0.0224, grad_fn=<MseLossBackward0>) tensor([ 0.1984, -0.2303]) tensor([-1.8898,  1.8998], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
40 tensor(0.0196, grad_fn=<MseLossBackward0>) tensor([ 0.2351, -0.1412]) tensor([-1.8918,  1.9021], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
40 tensor(0.0163, grad_fn=<MseLossBackward0>) tensor([ 0.1682, -0.1539]) tensor([-1.8942,  1.9035], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
40 tensor(0.0141, grad_fn=<MseLossBackward0>) tensor([ 0.1570, -0.1245]) tensor([-1.8959,  1.9050], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
40 tensor(0.0195

59 tensor(0.0012, grad_fn=<MseLossBackward0>) tensor([ 0.0434, -0.0432]) tensor([-1.9726,  1.9731], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
59 tensor(0.0010, grad_fn=<MseLossBackward0>) tensor([ 0.0397, -0.0358]) tensor([-1.9730,  1.9735], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
59 tensor(0.0014, grad_fn=<MseLossBackward0>) tensor([ 0.0480, -0.0595]) tensor([-1.9734,  1.9739], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
60 tensor(0.0012, grad_fn=<MseLossBackward0>) tensor([ 0.0563, -0.0369]) tensor([-1.9739,  1.9745], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
60 tensor(0.0010, grad_fn=<MseLossBackward0>) tensor([ 0.0405, -0.0404]) tensor([-1.9745,  1.9748], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
60 tensor(0.0009, grad_fn=<MseLossBackward0>) tensor([ 0.0370, -0.0335]) tensor([-1.9749,  1.9752], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
60 tensor(0.0012

76 tensor(0.0001, grad_fn=<MseLossBackward0>) tensor([ 0.0181, -0.0123]) tensor([-1.9916,  1.9913], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
76 tensor(0.0001, grad_fn=<MseLossBackward0>) tensor([ 0.0131, -0.0138]) tensor([-1.9918,  1.9914], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
76 tensor(9.9410e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0116, -0.0118]) tensor([-1.9919,  1.9916], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
76 tensor(0.0001, grad_fn=<MseLossBackward0>) tensor([ 0.0144, -0.0186]) tensor([-1.9920,  1.9917], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
77 tensor(0.0001, grad_fn=<MseLossBackward0>) tensor([ 0.0169, -0.0115]) tensor([-1.9922,  1.9919], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
77 tensor(0.0001, grad_fn=<MseLossBackward0>) tensor([ 0.0122, -0.0129]) tensor([-1.9923,  1.9920], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
77 tensor(8.

94 tensor(1.2465e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0040, -0.0053]) tensor([-1.9977,  1.9975], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
95 tensor(1.0474e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0049, -0.0031]) tensor([-1.9978,  1.9976], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
95 tensor(1.0662e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0035, -0.0039]) tensor([-1.9978,  1.9976], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
95 tensor(9.4385e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0028, -0.0036]) tensor([-1.9979,  1.9976], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
95 tensor(1.1057e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0037, -0.0050]) tensor([-1.9979,  1.9977], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
96 tensor(9.3126e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0045, -0.0028]) tensor([-1.9979,  1.9977], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([

In [11]:
parameters_to_vector(model.parameters())

tensor([-1.9984,  1.9983], grad_fn=<CatBackward0>)

In [12]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [13]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[j][model.user_id] - projected_weights[model.user_id][j]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [14]:
# Preparing projection matrices
models = [MLP_Net(user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, first_run=True):
    #projected_weights = []
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = parameters_to_vector(models[j].parameters()).size()[0], parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        mat.fill_diagonal_(1.0 + 1.0 * float(np.random.randn(1)))
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[j].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[j][i], parameters_to_vector(models[j].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights)



In [15]:
print(projection_list[0])

[0, tensor([[1.2305, 0.0000],
        [0.0000, 1.2305]]), tensor([[-1.0408,  0.0000],
        [ 0.0000, -1.0408]]), 0, 0, tensor([[0.7629, 0.0000],
        [0.0000, 0.7629]]), tensor([[-0.3075,  0.0000],
        [ 0.0000, -0.3075]]), 0, 0, 0, 0, 0, 0, 0, tensor([[1.6406, 0.0000],
        [0.0000, 1.6406]]), 0, 0, 0, 0, 0]


In [16]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [17]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [18]:
torch.norm(datapoints[model.user_id]['exact_weights'])

tensor(2.8284, dtype=torch.float64)

In [19]:
model = MLP_Net(user_id=0)

from torch.nn.utils import parameters_to_vector, vector_to_parameters

with torch.no_grad():    
    params = parameters_to_vector(model.parameters())

    print(params)

params *= 2.

vector_to_parameters(parameters=model.parameters(), vec=params)

parameters_to_vector(model.parameters())





tensor([-0.1144,  0.0210])


tensor([-0.2289,  0.0420], grad_fn=<CatBackward0>)

In [20]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    # Update prjection matrix
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            mat_vec_sum = torch.zeros_like(weights)
            for k in G.neighbors(i):
                mat_vec_sum = torch.add(mat_vec_sum, projected_weights[k][i] - projected_weights[i][k])
            #print(torch.outer(mat_vec_sum, weights))

            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * mat_vec_sum)
                                         
    projected_weights = []                                          
    update_ProjWeight(projection_list, projected_weights, first_run=False)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f,   Relative Error %2.5f"% (test_loss[-1], total_rel_error[-1]))

  0%|          | 0/2000 [00:00<?, ?it/s]

Training_loss 5.54895,   Relative Error 0.89737


  0%|          | 2/2000 [00:00<03:04, 10.82it/s]

Training_loss 5.52914,   Relative Error 0.89577
Training_loss 5.51639,   Relative Error 0.89474


  0%|          | 4/2000 [00:00<02:49, 11.77it/s]

...

100%|█████████▉| 1997/2000 [03:38<00:00,  9.43it/s]

Training_loss 0.00528,   Relative Error 0.02775
Training_loss 0.00525,   Relative Error 0.02767


100%|██████████| 2000/2000 [03:38<00:00,  9.44it/s]

Training_loss 0.00522,   Relative Error 0.02760
Training_loss 0.00520,   Relative Error 0.02755
Training_loss 0.00518,   Relative Error 0.02749


100%|██████████| 2000/2000 [03:38<00:00,  9.15it/s]


In [21]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

tensor([-1.9453,  1.9448], grad_fn=<CatBackward0>)

In [22]:
for j in G.neighbors(0):
    print(j)

1
2
5
6
14


In [23]:
parameters_to_vector(models[0].parameters())

tensor([1.9523, 1.9518], grad_fn=<CatBackward0>)

In [24]:
projection_list[0]

[0,
 tensor([[1.2365, 0.0207],
         [0.0060, 1.2512]]),
 tensor([[-1.0348,  0.0207],
         [ 0.0060, -1.0201]]),
 0,
 0,
 tensor([[0.7689, 0.0207],
         [0.0060, 0.7836]]),
 tensor([[-0.3015,  0.0207],
         [ 0.0060, -0.2868]]),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 tensor([[1.6466, 0.0207],
         [0.0060, 1.6613]]),
 0,
 0,
 0,
 0,
 0]

In [25]:
projected_weights[0]

[0,
 tensor([1.6097, 1.6015]),
 tensor([5.4405, 5.3694]),
 0,
 0,
 tensor([0.1915, 0.1914]),
 tensor([5.7820, 5.7552]),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 tensor([-5.0619,  5.0777]),
 0,
 0,
 0,
 0,
 0]

In [26]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

In [27]:
print(test_loss)

[5.54894900e+00 5.52913618e+00 5.51638758e+00 ... 5.22315688e-03
 5.20444824e-03 5.18068019e-03]


In [28]:
np.save( 'training_loss_sheave_fml', test_loss)
np.save('relative_error_sheave_fml', total_rel_error)